# Diferenciación automática

<a href="https://colab.research.google.com/github/milocortes/mod_04_concentracion/blob/ccm-2024/src/notebooks/python/numeros_duales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* [Algorithms for Optimization, sección 2.4](https://algorithmsbook.com/optimization/files/optimization.pdf)
* [Introduction to Automatic Differentiation and MATLAB Object-Oriented Programming](https://epubs.siam.org/doi/10.1137/080743627)

La [diferenciación automática](https://en.wikipedia.org/wiki/Automatic_differentiation) es un método para evaluar derivadas de funciones representadas como programas usualamente conocido como gráfica de cómputo.[[Automatic Differentiation in Machine Learning: a Survey, Baydin et. al, 2018](https://arxiv.org/abs/1502.05767)]. El programa está compuesto por operaciones elementales como sumas, restas, multiplicaciones y divisiones. 

Una gráfica de cómputo representa una función donde los nodos son operaciones y las aristas son relaciones de entrada-salida. Los nodos hoja de una gráfica computacional son variables de entrada o constantes, y los nodos terminales son valores de salida de la función. 

Hay dos métodos de diferenciación automática usando una gráfica de cómputo.

* **Forward accumulation** : el método usa **números duales** para recorrer el árbol desde las entradas hasta las salidas.
* **Backward accumulation** : recorre el árbol de las salidas a las entradas.

## Forward accumulation

Para la función:

$$
    \begin{equation}
        6x^2 - 12x +3
    \end{equation}
$$

La gráfica de cómputo asociada a esta función es la siguiente:

Para calcular el valor de la función y de su derivada en el punto $x=3$, recorreríamos la gráfica de la siguiente manera:

![Diferenciación automática](compu_graph_one_dim.png)

La gráfica de cómputo tomaría la siguiente configuración inicial:

![Diferenciación automática. Configuración inicial](config_compu_graph_one_dim.png)


## Implementación de números duales

Todos los datos en un programa Python están representados por objetos o por relaciones entre objetos (ver : https://docs.python.org/3/reference/datamodel.html).

Nosotros podemos modificar estas relaciones entre objetos al modificar los denominados "atributos especiales".

Estos son atributos proporcionan acceso a la implementación de operaciones entre clases y podemos modificarlos 
(sobrecargarlos).

Estos atributos especiales inicial con **__** y terminan de la misma forma con **__**.

Para la implementación de números duales nos interesan los atributos especiales que pueden ser definidos 
para emular objetos numéricos y operaciones entre estos objetos numéricos.

Por ejemplo, para evaluar la expresión x + y, donde x es una instancia de una clase que tiene el método especial
**__**add**__()**,  se llama al método x.**__**add**__**(x,y).

Por ejemplo, las siguientes operaciones son equivalentes:
* 4.7 + 6.8
* 4.7.**__**add**__**(6.8)
    
Con el objetivo de implementar el método Forward Accumulation de diferenciación automática, 
el cual diferencía una función al recorrer "hacia adelante" la gráfica computacional de la función (
Una gráfica computacional representa una función donde los nodos representan operaciones y las aristas relaciones 
de entrada-salida).

El recorrido a través de la gráfica computacional puede ser automatizado al sobrecargar cada operación aritmética que  produce el valor de la función y de su derivada. Esto es, tenemos que sobrecargar las reglas de derivación para la suma, resta, división, regla de la cadena, etc.


In [ ]:

class Dual:
    """
    Implementación de números duales.
    
    ...

    Atributos
    ----------
    val : float
        valor de la variable a evaluar en la función
    der : float
        valor de la dericaba de la variable a evaluar en la función

    Métodos
    -------
    __add__()
        sobrecarga la operación aritmética de suma
    
    __sub__()
        sobrecarga la operación aritmética de resta
        
    """
    
    def __init__(self, val, der):
        self.val = val
        self.der = der
    
    def __repr__(self):
        return f"Valor: {self.val}, Derivada: {self.der}"
    
    ## Sobrecargamos el método de suma al incorporar las reglas de derivación para la suma. 
    
    ## Para f(x) y g(x) 
    def __add__(self, other):
        if isinstance(other, Dual):
            return Dual(self.val + other.val, # f(x) + g(x)
                        self.der + other.der) # fʼ(x) + gʼ(x)
        else:
            ## El otro objeto es una constante, no un número dual
            return Dual(self.val + other, # f(x) + c
                        self.der + 0) # fʼ(x) + 0

    ## Para g(x) y f(x) 
    def __radd__(self, other):
        if isinstance(other, Dual):
            return Dual(self.val + other.val # g(x) + f(x)  
                        , self.der + other.der) # gʼ(x) + fʼ(x)  
        else:
            return Dual(self.val + other, #  c + f(x)
                        self.der + 0) # 0 + fʼ(x)

    ## Sobrecargamos el método de resta al incorporar las reglas de derivación para la resta. 
    
    ## Para f(x) y g(x) 
    def __sub__(self, other):
        if isinstance(other, Dual):
            return Dual(self.val - other.val, # f(x) - g(x)
                        self.der - other.der) # fʼ(x) - gʼ(x)
        else:
            ## El otro objeto es una constante, no un número dual
            return Dual(self.val - other, # f(x) - c
                        self.der - 0) # fʼ(x) - 0

    def __rsub__(self, other):
        if isinstance(other, Dual):
            return Dual(self.val - other.val, # g(x) - f(x)
                        self.der - other.der) # gʼ(x) - fʼ(x)
        else:
            return Dual(self.val - other, #  c - f(x)
                        self.der - 0) # 0 - fʼ(x)

    ## Sobrecargamos el método de multiplicación al incorporar las reglas de derivación para la multiplicación. 
    def __mul__(self, other):
        if isinstance(other, Dual):
            return Dual(self.val * other.val, # f(x) * g(x)
                        self.val * other.der + other.val * self.der ) # f(x) * gʼ(x) + g(x) * fʼ(x)  
        else:
            return Dual(self.val * other, # f(x) * c
                        self.der * other) # fʼ(x) * c

    def __rmul__(self, other):
        if isinstance(other, Dual):
            return Dual(self.val * other.val, # g(x) * f(x)  
                        self.val * other.der + other.val * self.der ) # g(x) * fʼ(x) + f(x) * gʼ(x)   
        else:
            return Dual(self.val * other, self.der * other)

    ## Sobrecargamos el método de potencia
    def __pow__(self, a):
        return Dual(self.val**a, # f(x)^a
                    a* self.val**(a-1)) # a f(x)^(a-1)

    ## Sobrecargamos el método de división
    def __truediv__(self, other):
        return Dual(self.val / other.val, # f(x)/g(x)
                    (self.der*other.val - other.der*self.val)/(other.val)**2) # fʼ(x)*g(x) - gʼ(x)*f(x) / (g(x)^2)

## Probemos la clase <code>Dual</code>

### Función a)
Para la función:

$$
    \begin{equation}
        6x^2 - 12x +3
    \end{equation}
$$

¿Cual es el valor de la función y de su derivada en el punto $x = 3$?

In [ ]:
# Generamos un objeto Dual. El primer argumento del constructor es el valor de la variable a evaluar 
# y el segunfo argumento indica que se requiere evaluar en la derivada de la función.
x = Dual(3,1)

6*x**2 - 12*x + 3


### Función b)

Para la función:

$$
    \begin{equation}
        \dfrac{x^2 + 3x}{x^3 + x^2 +2}
    \end{equation}
$$

¿Cual es el valor de la función y de su derivada en el punto $x = 3$?



In [ ]:
(x**2 + x*3)/(x**3 + x**2 + 2)


In [ ]:
# Verifiquemos el resultado anterior
def f(x):
    return (x**2 + x*3)/(x**3 + x*2 + 2)

def df(x):
    return (((2*x + 3)*(x**3 + x*2 + 2)) - ((x**2 + x*3)*(3*x**2 + 2)))/(x**3 + x*2 + 2)**2

In [ ]:
f(3)

In [ ]:
df(3)

### Función c)

Para la función:

$$
    \begin{equation}
        (4x^2 + 16x +3)^2
    \end{equation}
$$

¿Cual es el valor de la función y de su derivada en el punto $x = 3$?

Tenemos que implementar la regla de la cadena de potencias para números duales.

In [ ]:
def chain_rule_power(dual_number, power):
    return Dual(dual_number.val**power, # f(g(x))^a
                power*(dual_number.val) * (dual_number.der) ) # a * f(x) *gʼ(x)

In [ ]:
chain_rule_power(4*x**2 + 16*x +3, 2)


In [ ]:
# Verifiquemos el resultado anterior
def f(x):
    return (4*x**2 + 16*x + 3)**2 

def df(x):
    return 2*(4*x**2 + 16*x +3)*(8*x + 16)

In [ ]:
f(3)

In [ ]:
df(3)

## Gradiente con números duales

Hasta el momento sólo hemos trabajado con funciones de una variable. Para extender el método de diferenciación automática con *forward accumulation* para funciones de varias variables, necesitamos agregar el cálculo del gradiente con números duales.

Cuando consideramos funciones de varias variables, la gráfica de cómputo es prácticamente la misma que para el caso univariado, de manera que podemos extender el método *forward accumulation* sin grandes complicaciones. 

Considerando la función 

$$f(x,y) = 2x^3 + 3y^2 + c$$

La gráfica de cómputo asociada es:

<center><img src="https://raw.githubusercontent.com/bereml/iap/master/fig/autodiff_example.svg" width="800" align="center"/></center>
&nbsp;

[Tomado del ejemplo de Bere y Ricardo](https://github.com/gibranfp/CursoAprendizajeProfundo/blob/2022-1/notebooks/1c_pytorch.ipynb)

In [ ]:
import numpy as np

def gradiente(f, *xs):
    gs = []

    for i,x in enumerate(xs):
        if i == 0:
            gs.append(f(Dual(x, 1), *xs[1:]))
        else:
            gs.append(f(*xs[:i], Dual(x,1), *xs[(i+1):]))

    return np.array([g.der for g in gs])

### Función d)

Para la función:

$$
    \begin{equation}
        3x^2 + 10y^2
    \end{equation}
$$

¿Cual es el gradiente en el punto $(x,y) = (2,3)$?



In [ ]:
def f(x,y):
    return 3*x**2 + 10*y**2

gradiente(f, 2, 3)

### Función e) Himmelblau

$$
    \begin{equation}
        f(x,y) = (x^2 + y -11)^2 + (x + y^2 -7)^2
    \end{equation}
$$

con gradiente


$$
\nabla f(x,y)=  \begin{bmatrix}
\frac{\partial f(x,y)}{\partial x} \\
\frac{\partial f(x,y)}{\partial y}
\end{bmatrix} =
 \begin{bmatrix}
4x (x^2 + y - 11) + 2(x + y^2 - 7) \\
2(x^2 + y - 11) + 4  y  (x + y^2 - 7)
\end{bmatrix} 
$$

¿Cual es el gradiente en el punto $(x,y) = (-3,3)$?


In [ ]:
def himmelblau_dual(x,y):
    return chain_rule_power(x**2 + y -11, 2) + chain_rule_power(x + y**2 -7, 2)

In [ ]:
gradiente(himmelblau_dual, -3, 3)

In [ ]:
def himmelblau(x,y):
    return (x**2 + y -11)**2 + (x + y**2 -7)**2

def gradiente_himmelblau(x,y):
    dx = 4 * x * (x**2 + y - 11) + 2 * (x + y**2 - 7)
    dy = 2 * (x**2 + y - 11) + 4 * y * (x + y**2 - 7)
    return np.array([dx,dy])

xk_himm = [-3,3]

himmelblau(*xk_himm)
gradiente_himmelblau(*xk_himm)


## Algoritmo de descenso de gradiente con números duales

In [ ]:
# Definimos punto inicial de búsqueda
x0 = np.array([-3,3])

# Definimos la tasa de aprendizaje
rho = 0.005

# Definimos la cantidad de iteraciones
n = 40

xt = x0

# Comenzamos las iteraciones
for it in range(n):
    print(f"Iteración {it}. Evaluación del punto {xt} en la función: {himmelblau(*xt)}")

    ## Actualizamos xt con la regla de actualización de deepest descent
    xt = xt - rho*gradiente(himmelblau_dual, *xt) 


## Función f) McCormick

$$
    \begin{equation}
        f(x,y) = \sin(x+y) + (x-y)^2 - 1.5x + 2.5y + 1
    \end{equation}
$$


In [ ]:
import math


def chain_rule_sin(dual_number):
    return Dual(math.sin(dual_number.val), # sin(f(x))
                math.cos(dual_number.val) * (dual_number.der) ) # cos(f(x)) * fʼ(x)

def McCormick(x,y):
    return math.sin(x+y) + (x-y)**2 - 1.5*x + 2.5*y + 1

def McCormick_dual(x,y):
    return chain_rule_sin(x+y) + chain_rule_power(x-y,2) - 1.5*x + 2.5*y + 1

In [ ]:
# Definimos punto inicial de búsqueda
x0 = np.array([-3,3])

# Definimos la tasa de aprendizaje
rho = 0.005

# Definimos la cantidad de iteraciones
n = 2000

xt = x0

# Comenzamos las iteraciones
for it in range(n):
    
    if it % 100 == 0:
        print(f"Iteración {it}. Evaluación del punto {xt} en la función: {McCormick(*xt)}")

    ## Actualizamos xt con la regla de actualización de deepest descent
    xt = xt - rho*gradiente(McCormick_dual, *xt) 